In [1]:
import pandas as pd
import numpy as np

In [2]:
click_columns = ['user_id','item_id','time']
train_click_T = pd.DataFrame()
test_click_T = pd.DataFrame()
test_qtime_T = pd.DataFrame()


# 第t个阶段，读取所有文件并合并
t = 4
for i in range(t):
    temp_train_click = pd.read_csv('../data/underexpose_train/underexpose_train_click-%s.csv'%(str(i)),sep=',',names = click_columns )
    temp_test_click = pd.read_csv('../data/underexpose_test/underexpose_test_click-%s.csv'%(str(i)),sep=',',names = click_columns)
    temp_test_qtime = pd.read_csv('../data/underexpose_test/underexpose_test_qtime-%s.csv'%(str(i)),sep=',',names = ['user_id','time'])
    train_click_T = train_click_T.append(temp_train_click)
    test_click_T = test_click_T.append(temp_test_click)
    test_qtime_T = test_qtime_T.append(temp_test_qtime)

# 拼接所有历史点击数据
data_click_T = pd.concat([train_click_T,test_click_T],axis = 0).reset_index(drop = True)


# 计算所有商品的流行度-历史点击次数排序
data_click_T_item = data_click_T.groupby('item_id',as_index= False)['user_id'].agg({'user_dis':'unique','click_cnt':'count'})
data_click_T_item_rank = data_click_T_item.sort_values(by = 'click_cnt',axis = 0,ascending = False)['item_id'].tolist()

In [3]:
whole_dataset = pd.concat([train_click_T,test_click_T]).reset_index(drop=True)

In [4]:
whole_dataset

,user_id,item_id,time
0,4965,18,0.983763
1,20192,34,0.983772
2,30128,91,0.983780
3,29473,189,0.983930
4,10625,225,0.983925
...,...,...,...
1084190,34323,116409,0.983952
1084191,23433,116550,0.984105
1084192,11751,116908,0.984015
1084193,12565,117502,0.984052


In [8]:
count_item = whole_dataset.groupby('user_id').count()

In [9]:
count_item.columns = ['item_count','time_count']
count_item['user_id'] = count_item.index

count_item

,item_count,time_count,user_id
user_id,,,
1,49,49,1
2,79,79,2
3,36,36,3
4,9,9,4
6,17,17,6
...,...,...,...
35422,28,28,35422
35424,10,10,35424
35430,4,4,35430


In [18]:
count_item.groupby('item_count').count().sort_values('user_id',ascending=False).head(10)

,time_count,user_id
item_count,,
3,933,933
7,744,744
6,723,723
4,707,707
8,692,692
9,676,676
10,618,618
11,604,604
12,575,575


In [15]:
count_item.groupby('item_count').count().mean()

time_count    76.605882
user_id       76.605882
dtype: float64